In [ ]:
import logging
import pathlib
import typing
import json
import seaborn as sns
import glob

import cltrier_lib as lib
import pandas as pd

import llm_moral_values as llmv

logging.getLogger().setLevel(logging.INFO)

In [2]:
PERSONA_ORDER = ["argentine", "belgian"]
MODEL_ORDER = [model.name for model in llmv.schemas.Model.from_inference_selection()]

In [7]:
data = (
    pd.concat(
        [
            pd.json_normalize(json.load(open(file)))
            for file in glob.glob("./data/**/*.json", recursive=True)
        ]
    )
    .pipe(
        lambda _df: _df.assign(
            segment=_df["segment"].astype("category"),
            model=pd.Categorical(_df["model"].str.split("-").str[0]),
            persona=pd.Categorical(_df["persona"]),
            response=pd.to_numeric(_df["response"]),
            id=pd.to_numeric(_df["id"]),
            dimension=_df["dimension"].astype("category"),
        )
    )
    .sort_index()
    .dropna()
)


with open("survey.report.txt", "w") as f:
    for model, group in (
        data.groupby(["segment", "id", "model", "persona"], observed=True)["response"].var().groupby("model", observed=False)
    ):
        f.write(f"{model:-^42}\n")
        f.write(f"answers w/o variance: {len(group[group == 0.0])}/{len(group)}\n")
        f.write(f"mean variance: {group.mean():2.3f}\n")
        f.write("answers with variance (10 decile):\n")
        f.write(f"{group[group > group.quantile(.9)].droplevel(2).sort_values(ascending=False).to_string()}\n")
        f.write("\n\n")

In [ ]:
grid = sns.FacetGrid(
    (
        self.data[self.data["dimension"] != "catch"]
        .pipe(lambda _df: _df.assign(dimension=_df["dimension"].cat.remove_unused_categories()))
        .reset_index()
    ),
    col="model",
    col_wrap=4,
    height=4,
)

grid.map_dataframe(
    sns.pointplot, x="response", y="dimension", hue="persona", errorbar="sd", **self.args.plot_styles
)
grid.add_legend()
grid.savefig(export_path)

In [3]:
llmv.pipelines.ConductSurvey(
    iterations=1,
    models=llmv.schemas.Model.from_inference_selection()[:3],
    survey=llmv.questionnaire.Survey(path=pathlib.Path("../../data/moral_foundations_2")),
    personas=llmv.schemas.Persona.from_directory(pathlib.Path("./personas/")),
    export_path=pathlib.Path("./data/"),
)()

INFO:root:> Conducting Survey
INFO:root:Generated 1 surveys for configuration: llama3.1:8b:belgian
INFO:root:Generated 1 surveys for configuration: llama3.1:8b:argentine
INFO:root:Generated 1 surveys for configuration: llama3.3:70b:belgian
INFO:root:Generated 1 surveys for configuration: llama3.3:70b:argentine
INFO:root:Generated 1 surveys for configuration: mistral:7b:belgian
INFO:root:Generated 1 surveys for configuration: mistral:7b:argentine
INFO:root:> Collate Data
INFO:root:> Write Data Report
INFO:root:> Generate Cross Evaluation


StopIteration: 

In [ ]:
dataset: llmv.data.Survey = llmv.data.Survey(data=pd.read_parquet("./data/survey.parquet"))
dataset.data

In [ ]:
dataset.data.groupby(["model", "persona"], observed=True).size()

In [ ]:
(dataset.data.groupby(["model", "persona", "dimension"], observed=True).mean().squeeze(axis=1))

In [ ]:
dataset.plot("./report/points.by.model_persona_dimension.pdf")

In [ ]:
cross_evaluation: llmv.data.CrossEvaluation = llmv.data.CrossEvaluation(
    data=pd.read_parquet("./data/cross_evaluation.parquet")
)
cross_evaluation.data

In [ ]:
cross_evaluation.plot("./report/heat.cross_evaluation.pdf")